In [11]:
!pip uninstall opencv-python-headless torch mediapipe pandas opencv-python

^C


In [ ]:
!pip uninstall torch

In [2]:
import cv2
import torch
import mediapipe as mp

# Step 1: Load YOLOv8 model for object detection
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Pre-trained YOLO model

# Step 2: Initialize MediaPipe Pose for pose estimation
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Step 3: Define helper functions
def detect_cyclists_and_runners(image):
    """
    Detect cyclists and runners in the input image using YOLO and MediaPipe Pose.
    Args:
        image: Input image.
    Returns:
        Annotated image with cyclists and runners highlighted.
    """
    # Convert image to RGB for MediaPipe
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Step 3.1: Run YOLO detection
    results = model(rgb_image)
    detections = results.pandas().xyxy[0]  # YOLO detections as DataFrame

    for _, detection in detections.iterrows():
        # Extract bounding box and label
        xmin, ymin, xmax, ymax, label, confidence = (
            int(detection['xmin']),
            int(detection['ymin']),
            int(detection['xmax']),
            int(detection['ymax']),
            detection['name'],
            detection['confidence'],
        )

        # Crop the detected person
        if label == 'person':
            person_roi = rgb_image[ymin:ymax, xmin:xmax]

            # Step 3.2: Use MediaPipe Pose for pose estimation
            results_pose = pose.process(person_roi)
            if results_pose.pose_landmarks:
                # Analyze pose to classify as running or cycling
                annotated_image = analyze_pose(image, xmin, ymin, xmax, ymax, results_pose.pose_landmarks)

    return image

def analyze_pose(image, xmin, ymin, xmax, ymax, landmarks):
    """
    Analyze pose landmarks to determine if the person is running or cycling.
    Args:
        image: Original image.
        xmin, ymin, xmax, ymax: Bounding box coordinates.
        landmarks: Pose landmarks.
    Returns:
        Annotated image with activity labeled.
    """
    # Extract key landmarks (e.g., knees, ankles, wrists)
    left_knee = landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE]
    right_knee = landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE]
    left_ankle = landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE]
    right_ankle = landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE]

    # Example heuristic for classification:
    # - Cycling: Knees move in circular motion, feet near pedals (higher vertical range)
    # - Running: High vertical motion of knees, regular stride motion

    knee_distance = abs(left_knee.y - right_knee.y)
    ankle_distance = abs(left_ankle.y - right_ankle.y)

    label = "Running" if knee_distance > 0.2 else "Cycling"

    # Draw the bounding box and label
    cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
    cv2.putText(image, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    return image

# Step 4: Run on a sample image
if __name__ == "__main__":
    # Load input image
    image_path = "street_scene.jpg"  # Replace with your image path
    image = cv2.imread(image_path)

    # Detect cyclists and runners
    output_image = detect_cyclists_and_runners(image)

    # Display the output
    cv2.imshow("Output", output_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'cv2'

In [7]:
!where python

C:\Users\sangouda\AppData\Local\Programs\Python\Python311\python.exe
C:\Users\sangouda\AppData\Local\Microsoft\WindowsApps\python.exe
